# **Dragon Estate Limited**

In [ ]:
import pandas as pd
import numpy as np


: 

In [ ]:
housing = pd.read_csv("Data.csv")

: 

In [ ]:
housing.head()

: 

In [ ]:
housing.info()

: 

In [ ]:
housing['CHAS'].value_counts()

: 

In [ ]:
housing.describe()


: 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
#housing.hist(bins = 50 , figsize=(20,15))

: 

# **Train-Test Split**

In [ ]:
#def split_train_test(data,test_ratio):
    # np.random.seed(42)
    # suffled = np.random.permutation(len(data))
    # #print(suffled)
    # test_set_size = int(len(data) * test_ratio)
    # test_indices = suffled[:test_set_size]
    # train_indices = suffled[test_set_size:]
    # return data.iloc[test_indices], data.iloc[train_indices]

: 

In [ ]:
#test_set,train_set = split_train_test(housing,0.2)

: 

In [ ]:
#print(f"Rows in train set: {len(train_set)}\nRows in test set:{len(test_set)}")

: 

In [ ]:
from sklearn.model_selection import train_test_split
train_set,test_set = train_test_split(housing,test_size = 0.2,random_state = 42)


: 

# **Specific Selection**

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits = 1,test_size = 0.2,random_state = 42)
for train_index,test_index in split.split(housing,housing['CHAS']):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

: 

In [ ]:
strat_test_set['CHAS'].value_counts()

: 

In [ ]:
strat_train_set['CHAS'].value_counts()

: 

In [ ]:
#95/7

: 

In [ ]:
#376/28

: 

: 

# **Looking For Correlation**

In [ ]:
corr_matrix = housing.corr()
corr_matrix['MEDV'].sort_values(ascending = False)

: 

In [ ]:
from pandas.plotting import scatter_matrix
attr = ['MEDV','RM','ZN','LSTAT']
scatter_matrix(housing[attr])

: 

In [ ]:
housing.plot(kind='scatter',x = 'RM',y = 'MEDV',alpha = 0.8)

: 

In [ ]:
housing = strat_train_set.copy()

: 

: 



# **Trying out new Correlations**



In [ ]:
#housing["TAXRM"] = housing["TAX"]/housing["RM"]

: 

In [ ]:
#housing.head()


: 

In [ ]:
corr_matrix = housing.corr()
corr_matrix['MEDV'].sort_values(ascending = False)

: 

In [ ]:
#housing.plot(kind='scatter',x = 'TAXRM',y = 'MEDV',alpha = 0.8)

: 

In [ ]:
housing_feature = strat_train_set.drop("MEDV", axis=1)
housing_label = strat_train_set["MEDV"].copy()

: 

# **Pipeline**

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
my_pipeline = Pipeline([
    ('imputer' ,SimpleImputer(strategy = "median")),
    ('std_scaler', StandardScaler())
])

: 

In [ ]:
housing_num_tr = my_pipeline.fit_transform(housing_feature)

: 

In [ ]:
housing_num_tr.shape

: 

# **Selecting A Desired Model**

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
#model = LinearRegression()
#model = DecisionTreeRegressor()
model = RandomForestRegressor()
model.fit(housing_num_tr,housing_label)

: 

In [ ]:
some_data = housing_feature.iloc[:5]
some_label = housing_label.iloc[:5]

: 

In [ ]:
prepared_data = my_pipeline.transform(some_data)

: 

In [ ]:
model.predict(prepared_data)

: 

In [ ]:
list(some_label)

: 

# **Evaluating Model**

In [ ]:
from sklearn.metrics import mean_squared_error
housing_predictions = model.predict(housing_num_tr)
lin_mse = mean_squared_error(housing_label,housing_predictions)
lin_rmse = np.sqrt(lin_mse)

: 

In [ ]:
lin_rmse
# Model has Overfitted

: 

Using Better evalution technique - Cross validation

In [ ]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(model,housing_num_tr,housing_label,scoring = "neg_mean_squared_error",cv = 10)
rmse_score = np.sqrt(-score)

: 

In [ ]:
rmse_score

: 

# ***Saving the Model***

In [ ]:
from joblib import dump,load
dump(model,'Dragon.joblib')

: 

# ***Testing The Model On Test Data***

In [ ]:
X_test = strat_test_set.drop("MEDV",axis = 1)
Y_test = strat_test_set["MEDV"].copy()
X_prepared = my_pipeline.transform(X_test)
final_prediction = model.predict(X_prepared)
final_mse = mean_squared_error(Y_test,final_prediction)
final_rmse = np.sqrt(final_mse)


: 

In [ ]:
final_rmse

: 

: 